# Calcul du jacobien de la contrainte sur l'altitude finale du profil

Le profil du projet est modélisé par des segments de droite reliés entres eux par des cercles. 
La nouvelle paramétrisation de ce profil est une suite de triplet. Un triplet contient les informations suivantes :
* l'abscisse du début du segment de droite exprimé en mètre
* la pente du segment de droite exprimé en pourcent
* le rayon du cercle qui permet de relier le segment précédent au segment actuel exprimé en mètre

Cette construction est visible sur [ce schéma](https://ggbm.at/SeDfCnk3)

Le premier triplet est particulier : il comprend l'altitude de départ, l'abscisse de départ, et la pente de la droite
Le dernier triplet ne comprend qu'une information sur l'abscisse de fin.

Le profil initial est sur 8 810m, il commence à l'abscisse 0 et comprend 14 triplets.

L'optimisation va modifier une partie de ces triplets afin de minimiser l'énergie consommée lors de l'usage et lors de la construction. Ce dernier est calculé à partir du profil du terrain naturel qui est disponible de l'abscisse 1821 m à l'abscisse 6 495m. Ceci correspond à 6 triplets.

Les degrés de libertés qui seront utilisés par l'algorithme d'optimisation sont pour chaque triplet : 
* un décalage par rapport à l'abscisse de départ
* les pentes des différentes droites
* les rayons des différents cercles

la première abscisse est 1821.5145312001146 + X(1)
la deuxième abscisse est 1821.5145312001146 + X(1) + X(2)
...
la sixième abscisse est 1821.5145312001146 + X(1) + X(2) + X(3) +  X(4) + X(5) +  X(6)

les pentes sont donnée en pourcent et correspondent aux degrés de liberté : X(7)..X(12). X(7)=1 correspond à une pente de 0.01 pour la droite considéré

les rayons sont donnés en mètre et correspondent aux degrés de liberté X(13)..X(18)

Ainsi, il y a 18 degrés de libertés. Evidemment, ceux-ci sont contraints, les pentes sont limitées, la somme des décalage doit être inférieur à la fin du profil du terrain naturel connu soit 6495m, et l'altitude finale du profil
avec cette modélisation, l'altitude à l'abscisse finale 8 810m dépend des degrés de liberté. 

Or pour ce raccorder à la route, cette altitude doit être de 328,29 m.  Ceci ajoute une contrainte non linéaire d'égalité au problème. 

Nous nous sommes aperçu sur le programme d'optimisation de Matlab n'arrivait pas à trouver des solutions qui garantissent cette contrainte. Pour garantir cettte contrainte, Matlab calcule numériquementle jacobien de la contrainte. Or, nous avons approximé par une formule analytique ce jacobien et nous ne trouvions pas les mêmes valeurs. Matlab trouvait systématiquement un jacobien nul...

Notre hypothèse est que ceci provenait du codage des réels en 64 bits sous Matlab et en 32 bits sous Python. En effet, pour calculer le jacobien, Matlab fait varier de deux epsilon (epsilon est la précision du codage des réels) les degrés de libertés et calcule la différence en terme d'altitude finale.

Nous sommes passés à un codage en 64 bits sous Python. Ce document a pour objectif de vérifier que le calcul du jacobien est, maintenant, correct.

Nous avons au préalable vérifier avec le programme sous matlab test_passage_64bits_matlab_python.m que le codage sous Matlab et sous Python est le même.

Maintenant, nous allons vérifier le calcul symbolique et numérique du gradient.





## Calcul symbolique du gradient

Nous supposons que le raccordement par les cercles des segments de droite ne joue pas sur l'altitude. Ceci est vrai en théorie. En pratique, afin d'assurer que l'algorithme qui dessine le profile donne toujours un résultat, le segment de droite peut être complètement remplacé par l'arc de cercle et dans ce cas, celui-ci a une influence sur l'altitude. C'est un cas exceptionnel.

Sous cette hypothèse l'altitude  finale, $h_f$, se calcule avec la formule suivante

$$h_f = h_0 + \sum\limits_{i=1}^{13} p_i \times l_i$$

* $h_0$ est l'altitude initiale,
* $p_i$ est la pente de la droite, 
* $l_i$ est la longueur du segment de droite

Si nous exprimons cette équations en fonction des degrés de liberté X: 

$$h_f = h_0 + K  + X(1)  \times \frac{-1.883}{100} + X(2) \times \frac{X(7)}{100} + X(3) \times \frac{X(8)}{100} +\cdots+ X(6) \times \frac{X(11)}{100} +  \left(6495.31 - \left(1821.5145312001146 + X(1) + X(2) + X(3) +  X(4) + X(5) +  X(6) \right) \right) \times \frac{X(12)}{100}$$

* -1.883 est la pente en pourcent du segment droite avant l'abscisse de début du terrain naturel
* $h_0$ est l'altitude initiale égale à 375.42 m


Le gradient s'en déduit naturellement
  $$\frac{\partial h_f}{\partial X(1)}=\frac{-1.883}{100}-\frac{X(12)}{100}$$
   pour i allant de 2 à 6
  $$\frac{\partial h_f}{\partial X(i)}=\frac{X(i+5)}{100}-\frac{X(12)}{100}$$
   pour i allant de 7 à 11
   $$\frac{\partial h_f}{\partial X(i)}=\frac{X(i-5)}{100}$$ 
   $$\frac{\partial h_f}{\partial X(12)}=\left(6495.31 - \left(1821.5145312001146 + X(1) + X(2) + X(3) +  X(4) + X(5) +  X(6) \right) \right) /100$$
    pour i allant de 13 à 18
 $$\frac{\partial h_f}{\partial X(i)}=0$$

## Calcul numérique du gradient

    Dans la suite, nous allons comparer les valeurs des dérivées calculées numériquement et celles calculées symboliquement
    
    les deux premières cellules permettent d'initialiser le processe. Nous partons du profil modifié par michel puis nous ajoutons un incrément sur X(8), c'est à dire la pente 0.140 (5ème triplet) et comparons la valeur du gradient calculé numériquement avec la valeur calculée symboliquement à l'aide des formules du dessus.
    Nous faisons la même opération avec X(1), X(12) et X(17)

In [19]:
import numpy as np
import copy
import math
from raccordement_calcul_jacobien_contrainte import profil_rn, model_old2model_new


In [20]:
profil_modifie_michel=[[0.0,375.42, -5.215, 3015],
[711 , 352.51, -0.502, 13508],
[1762, 334.03, -1.883, 15127],
[2596, 324.39, -1.1, 20000],
[3405, 323.99, 0.140, 20000],
[3814, 321.20, -1.000, 30000],
[4804, 325.92, 1.230, 30000],
[5440, 326.99, -0.880, 29544],
[6457, 331.57, 0.923, 7789],
[6532, 332.61, 1.871, 10539],
[7260, 338.18, 0.656, 14622],
[8365, 334.87, -0.372, 14825],
[8810, 328.29, -1.843]]
profil_modifie_michel_new=model_old2model_new(profil_modifie_michel)
xxx,yyy=profil_rn(profil_modifie_michel_new)
print profil_modifie_michel_new

[[375.42, 0.0, -5.215], [410.37088903034316, -0.502, 3015], [805.8826937002143, -1.883, 13508], [1821.5146871008888, -1.1, 15127], [2719.5967741935397, 0.14, 20000], [3519.0350877192996, -1.0, 20000], [4148.3946188340797, 1.23, 30000], [5119.9620853080569, -0.88, 30000], [5706.6062118691116, 0.923, 29544], [6495.3175105485197, 1.871, 7789], [6597.3761316872433, 0.656, 10539], [7337.8793774319065, -0.372, 14622], [8475.2209381373232, -1.843, 14825], [8810]]


      Derivée par rapport à X(8), c'est à dire la pente 0.140 %

In [21]:
profil_modifie_michel_incremente=copy.deepcopy(profil_modifie_michel_new);
increment=0.01;
profil_modifie_michel_incremente[4][1]=profil_modifie_michel_incremente[4][1]+increment;
xxxinc,yyyinc=profil_rn(profil_modifie_michel_incremente)
deriveeX8=(yyyinc[-1]-yyy[-1])/increment
deriveeX8_symbolique=(profil_modifie_michel_incremente[5][0]-profil_modifie_michel_incremente[4][0])/100.00
print deriveeX8
print deriveeX8_symbolique

7.99438313526
7.99438313526


    Dérivée par rapport à X(1), c'est à dire que l'on ajoute un incrément à 1821.51 m

In [22]:
profil_modifie_michel_incremente=copy.deepcopy(profil_modifie_michel_new);
increment=0.01;
profil_modifie_michel_incremente[3][0]=profil_modifie_michel_incremente[3][0]+increment;
profil_modifie_michel_incremente[4][0]=profil_modifie_michel_incremente[4][0]+increment;
profil_modifie_michel_incremente[5][0]=profil_modifie_michel_incremente[5][0]+increment;
profil_modifie_michel_incremente[6][0]=profil_modifie_michel_incremente[6][0]+increment;
profil_modifie_michel_incremente[7][0]=profil_modifie_michel_incremente[7][0]+increment;
profil_modifie_michel_incremente[8][0]=profil_modifie_michel_incremente[8][0]+increment;
xxxinc,yyyinc=profil_rn(profil_modifie_michel_incremente)
deriveeX1=(yyyinc[-1]-yyy[-1])/increment
deriveeX1_symbolique=(profil_modifie_michel_incremente[2][1]-profil_modifie_michel_incremente[8][1])/100
print deriveeX1
print deriveeX1_symbolique

-0.0280599999996
-0.02806


    Dérivée par rapport à X(12), c'est à dire que l'on ajoute un incrément à 0.923 %


In [23]:
profil_modifie_michel_incremente=copy.deepcopy(profil_modifie_michel_new);
increment=0.01;
profil_modifie_michel_incremente[8][1]=profil_modifie_michel_incremente[8][1]+increment;
xxxinc,yyyinc=profil_rn(profil_modifie_michel_incremente)
deriveeX12=(yyyinc[-1]-yyy[-1])/increment
deriveeX12_symbolique=(profil_modifie_michel_incremente[9][0]-profil_modifie_michel_incremente[8][0])/100
print deriveeX12
print deriveeX12_symbolique

7.88711298679
7.88711298679


In [ ]:
profil_modifie_michel_incremente=copy.deepcopy(profil_modifie_michel_new);
increment=0.01;
profil_modifie_michel_incremente[8][1]=profil_modifie_michel_incremente[8][1]+increment;
xxxinc,yyyinc=profil_rn(profil_modifie_michel_incremente)
deriveeX12=(yyyinc[-1]-yyy[-1])/increment
deriveeX12_symbolique=(profil_modifie_michel_incremente[9][0]-profil_modifie_michel_incremente[8][0])/100
print deriveeX12
print deriveeX12_symbolique

    Dérivée par rapport à X17, c'est à dire que l'on ajoute un incrément à R= 29544m

In [25]:
profil_modifie_michel_incremente=copy.deepcopy(profil_modifie_michel_new);
increment=1;
profil_modifie_michel_incremente[7][2]=profil_modifie_michel_incremente[7][2]+increment;
xxxinc,yyyinc=profil_rn(profil_modifie_michel_incremente)
deriveeX12=(yyyinc[-1]-yyy[-1])/increment
deriveeX12_symbolique=0
print deriveeX12
print deriveeX12_symbolique

0.0
0
